In [7]:
import pandas as pd
import folium
import requests
import ast
import time
%run Tools.py
from math import cos, sin, atan2, sqrt, radians, degrees

def center_geolocation(geolocations): # 求多个坐标点的中心
    """
    输入多个经纬度坐标，找出中心点
    :param geolocations: 集合
    :return:
    """
    x = 0
    y = 0
    z = 0
    length = len(geolocations)
    for lat, lon in geolocations:
        lon = radians(float(lon))
        lat = radians(float(lat))
        x += cos(lat) * cos(lon)
        y += cos(lat) * sin(lon)
        z += sin(lat)

    x = float(x / length)
    y = float(y / length)
    z = float(z / length)

    return (degrees(atan2(z, sqrt(x * x + y * y)))), degrees(atan2(y, x))

def center_trajectory(sub_trajectorys): # 求多条轨迹的中心
    """
    输入多条子轨迹，找出中心轨迹
    :param sub_trajectorys: 列表
    :return:
    """

    CenTra = [] # 中心轨迹

    Len = [] # 同一类内所有子轨迹长度
    for i in range(len(sub_trajectorys)):
        Len.append(len(sub_trajectorys[i]))
    max_length = max(Len) # 最大子轨迹长度

    for j in range(max_length):
        geolocations = []
        for i in range(len(sub_trajectorys)):
            if len(sub_trajectorys[i])-1>=j:
                geolocations.append(sub_trajectorys[i][j])
        CenTra.append(center_geolocation(tuple(geolocations)))

    return CenTra

def georequest2(vin, geolocations): # 返回中心轨迹路段信息

    f = open('Address/trajectory/'+str(vin)+'.txt', 'a')
    Street = {} # 保存中心轨迹上所有轨迹点路段信息
    for i in range(len(geolocations)):
        # 接口请求地址
        url = "https://apis.map.qq.com/ws/geocoder/v1/"
        # 请求头
        headers = {
            "content-type": "application/json"
        }
        # 键值队
        payload = {
            "key": "FB4BZ-FA6L3-L4H3N-3LD53-FGZZO-YSBE3",
            "location": str(geolocations[i][0])+','+str(geolocations[i][1])
        }
        # 构造发送请求
        response = requests.get(url=url, headers=headers, params=payload)
        dict = ast.literal_eval(response.text)
        print(dict)
        f.writelines(str(dict["result"]["location"])+"\n") # 打印轨迹上各点坐标信息
        Street[dict["result"]["address_component"]["street"]] = Street.get(dict["result"]["address_component"]["street"],0)+1
    
    # 返回出现次数最多的路段信息
    max_time = 0
    street = None
    print(Street)
    for k in Street:
        if Street[k] >= max_time:
            max_time = Street[k]
            street = k

    # 打印路段信息
    f.writelines(str(street)+"\n")
    f.close()

In [2]:
df = pd.read_csv("SubTrajectory/wqmbpgahavumbbcba.csv",header=0)

Sub_Trajectory = [df[df["trajectory_name"]==0].reset_index(inplace=False,drop=True)] # 保存车主的全部子轨迹
Sub_Tra_Geo = [[]] # 用来保存各段子轨迹的经纬度坐标（只能保存一个车主的）
flag = 0 # 子轨迹名称
length = df.shape[0]
for i in range(length):
    if df["trajectory_name"][i]==flag:
        Sub_Tra_Geo[len(Sub_Tra_Geo)-1].append([df["latitude"][i],df["longitude"][i]])
    else:
        flag = flag+1
        Sub_Trajectory.append(df[df["trajectory_name"]==flag].reset_index(inplace=False,drop=True))
        Sub_Tra_Geo.append([[df["latitude"][i],df["longitude"][i]]])

result = DBSCAN_trajectory(Sub_Trajectory,0.02,5)  
"""
这一块的Sub_Tra_Geo和Sub_Trajectory的写法应该可以简化，有点乱
"""

'\n这一块的Sub_Tra_Geo和Sub_Trajectory的写法应该可以简化，有点乱\n'

In [8]:
map = folium.Map(location=Sub_Tra_Geo[0][0], zoom_start=15, attr='default', tiles='https://rt0.map.gtimg.com/tile?z={z}&x={x}&y={-y}')

midTra = [[] for i in range(max(result)+1)]
index = [[] for i in range(max(result)+1)]
length = len(Sub_Trajectory)
for j in range(length):
    if result[j] != -1:
        midTra[result[j]].append(Sub_Tra_Geo[j])
        index[result[j]].append(j)

for k in range(len(midTra)):
    centra = center_trajectory(midTra[k])
    print(centra)
    georequest2(Sub_Trajectory[0]["vin"][0], centra)
    folium.PolyLine(  
        centra,  
        weight=3,  
        color="red",  
        opacity=0.5
    ).add_to(map)
    map.save("Maps/轨迹聚类2.HTML")

[(41.69406373203524, 123.43997407760091), (41.69367959260376, 123.43978160785767), (41.69479846146359, 123.4397037899156)]
{'status': 0, 'message': 'query ok', 'request_id': '4ba1789c-9a4f-4599-a0dc-bbb3fa158a16', 'result': {'location': {'lat': 41.694064, 'lng': 123.439974}, 'address': '辽宁省沈阳市浑南区白塔三街', 'formatted_addresses': {'recommend': '浑南区理想新城(白塔三街东150米)', 'rough': '浑南区理想新城(白塔三街东150米)'}, 'address_component': {'nation': '中国', 'province': '辽宁省', 'city': '沈阳市', 'district': '浑南区', 'street': '白塔三街', 'street_number': '白塔三街'}, 'ad_info': {'nation_code': '156', 'adcode': '210112', 'city_code': '156210100', 'name': '中国,辽宁省,沈阳市,浑南区', 'location': {'lat': 41.714729, 'lng': 123.449597}, 'nation': '中国', 'province': '辽宁省', 'city': '沈阳市', 'district': '浑南区'}, 'address_reference': {'street_number': {'id': '', 'title': '', 'location': {'lat': 41.690243, 'lng': 123.437805}, '_distance': 170.4, '_dir_desc': '东'}, 'crossroad': {'id': '72777145', 'title': '白塔河路/白塔三街(路口)', 'location': {'lat': 41.69634, 'l

KeyError: 'result'